In [1]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import time

So the coverpage has to be scraped in 2 steps:   
```
1. All big stories have tag <div class = "media-heading">
2. Rest of the stories in the coverpage can be ientified by <a class = "wclink2">
```

In [3]:
df_features = pd.DataFrame(columns = ["title", "link", "Content"])

In [4]:
url = "https://www.hindustantimes.com/top-news/"

In [5]:
coverpage_request = requests.get(url)

In [6]:
coverpage_request.status_code

200

In [7]:
coverpage = coverpage_request.content

In [8]:
coverpage_soup = BeautifulSoup(coverpage, "html5lib")

In [9]:
#Extracting the big stories of the coverpage
#Using custom fuction to avoid capturing <div class= "media-heading headingfour"
bignews = coverpage_soup.find_all(lambda tag: tag.name == "div"
                                 and tag.get("class") == ["media-heading"])

In [10]:
links = [item.find("a")["href"] for item in bignews]
titles = [item.find("a").get_text() for item in bignews]

In [11]:
session = requests.session()

In [14]:
#testing code for one link before putting all links in a loop
article = session.get(links[0])

In [15]:
article_content = article.content

In [16]:
soup_article = BeautifulSoup(article_content, "html5lib")

In [17]:
body = soup_article.find("div", class_ = "storyDetail")

In [18]:
" ".join([p.get_text() for p in body.find_all("p")])
#end of testing

'The chief the Tablighi Jamaat, Maulana Saad, tells Hindustan\u2009Times  in an interview through his lawyer that he is willing to cooperate with the police, and that no illegal activity took place at the Markaz building, which is the biggest Covid-19 hot spot in India. Edited excerpts: Is it true that you organised Ijtema without permission? Do not confuse Ijtema with normal day-to-day activities and religious programmes held in Markaz Nizamuddin. Ijtema is generally a large congregation which is held in open public spaces for which proper permissions are taken from the concerned authorities.  Follow latest updates on coronavirus here  Markaz Nizamuddin is basically a masjid namely Bangle Wali Masjid, where normal religious sermons are given on a daily basis throughout the year. Being a masjid, I don’t think there is any requirement to seek permission to give sermons or to have other religious discourses within the building. Is it true that you’re not supporting the police investigati

In [19]:
# The philosophy is we fetch links one by one and only the links that can scraped and the corresponding titls will be added
#in the below lists
final_articles = []
final_links = []
final_titles = []

In [20]:
session = requests.session()

In [21]:
for n in np.arange(len(links)):
    time.sleep(1)
    if "/liveupdates.hindustantimes.com/" in links[n]:
        continue
    try:
        article = session.get(links[n])
        article_content = article.content
        soup_article = BeautifulSoup(article_content, "html5lib")
        body = soup_article.find("div", class_ = "storyDetail")
        final_article = " ".join([p.get_text() for p in body.find_all("p")])
        final_articles.append(final_article)
        final_titles.append(titles[n])
        final_links.append(links[n])
        print(n, "done")
    except:
        print(n, "failed")

0 done
1 done
2 done
3 done
4 done
5 done
6 done
7 done
8 done
9 done


In [23]:
df_bignews = pd.DataFrame({"title" : final_titles, "link" : final_links, "Content" : final_articles})

In [24]:
df_features = df_features.append(df_bignews, ignore_index = True)

In [25]:
df_features

,title,link,Content
0,‘Few tested positive’: Tablighi chief’s comeba...,https://www.hindustantimes.com/india-news/few-...,"The chief the Tablighi Jamaat, Maulana Saad, t..."
1,"Covid-19 not the last pandemic, world has to b...",https://www.hindustantimes.com/india-news/covi...,"Covid-19 is not the first, nor will it be the ..."
2,‘Unilateral decision’ to send team ‘undesirabl...,https://www.hindustantimes.com/india-news/unil...,On a day when Bengal recorded a sharp rise in ...
3,"WHO chief terms Covid-19 ‘Public Enemy No. 1’,...",https://www.hindustantimes.com/world-news/who-...,The World Health Organization chief warned Mon...
4,Three TV soaps will go off air after coronavir...,https://www.hindustantimes.com/tv/beyhadh-2-is...,"Three popular TV shows -- Beyhadh 2, Ishaaron ..."
5,"‘World not starting for next 1 year, how can c...",https://www.hindustantimes.com/cricket/world-i...,Former Pakistan pacer Shoaib Akhtar believes c...
6,WhatsApp introduces new group calling limits i...,https://www.hindustantimes.com/tech/whatsapp-i...,"WhatsApp, in the past couple of weeks has roll..."
7,Teacher climbs tree for internet connectivity ...,https://www.hindustantimes.com/education/teach...,"Prosperity is a great teacher, adversity even ..."
8,"Sensex tanks 800 points to 30,800 in opening s...",https://www.hindustantimes.com/business-news/s...,Indian equity indices opened in red on Tuesday...
9,UEFA look for light at end of tunnel in bid to...,https://www.hindustantimes.com/football/uefa-l...,As European countries take steps towards easin...


Extracting the rest of the articles on the coverpage

In [36]:
#Scraping rest of the articles on the coverpage
othernews = coverpage_soup.find_all("a", class_ = "wclink2")

In [37]:
len(othernews)

40

In [28]:
links = [item["href"] for item in othernews]
titles = [item.get_text() for item in othernews]

In [29]:
len(titles), len(links)

(40, 40)

In [30]:
#testing code for one link before putting all links in a loop
article = session.get(links[0])

In [ ]:
article_content = article.content

In [ ]:
soup_article = BeautifulSoup(article_content, "html5lib")

In [ ]:
body = soup_article.find("div", class_ = "storyDetail")

In [ ]:
" ".join([p.get_text() for p in body.find_all("p")])

In [31]:
# The philosophy is we fetch links one by one and only the links that can scraped and the corresponding titls will be added
#in the below lists
final_articles = []
final_links = []
final_titles = []

In [32]:
for n in np.arange(len(links)):
    #Accountinng for non-scrapable links
    if "/liveupdates.hindustantimes.com/" in links[n]:
        continue
    time.sleep(1)
    try:
        article = session.get(links[n])
        article_content = article.content
        soup_article = BeautifulSoup(article_content, "html5lib")
        body = soup_article.find("div", class_ = "storyDetail")
        final_article = " ".join([p.get_text() for p in body.find_all("p")])
        final_articles.append(final_article)
        final_titles.append(titles[n])
        final_links.append(links[n])
        print(n, "done")
    except:
        print(n, "failed")

0 done
1 done
3 done
4 done
5 done
6 done
7 done
8 done
9 done
10 done
11 done
12 done
13 done
14 done
15 done
16 done
17 done
18 done
19 done
20 done
21 done
22 done
23 done
24 done
25 done
26 done
27 done
28 done
29 done
30 done
31 done
32 done
33 done
34 done
35 done
36 done
37 done
38 done
39 done


In [33]:
df_othernews = pd.DataFrame({"title" : final_titles, "link" : final_links, "Content" : final_articles})

In [34]:
df_features = df_features.append(df_othernews, ignore_index=True)

In [5]:
df_features

,title,link,Content
0,"Delhi hits pause on MHA order to reopen shops,...",https://www.hindustantimes.com/india-news/delh...,The Delhi government is not inclined to implem...
1,Close contact of jamaat attendee tests positiv...,https://www.hindustantimes.com/india-news/clos...,A 40-year-old man tested positive for Covid-19...
2,Close contact of jamaat attendee tests positiv...,https://www.hindustantimes.com/india-news/clos...,A 40-year-old man tested positive for Covid-19...
3,Farmer’s selfless act of donating mask wells u...,https://www.hindustantimes.com/world-news/farm...,Help has poured into New York from many places...
4,"Hina Khan keeps her first roza, prays for ‘pro...",https://www.hindustantimes.com/tv/hina-khan-ke...,Actors from Bollywood and TV industries wished...
5,"Kohli, de Viliiers pick combined Ind-SA XI,Dho...",https://www.hindustantimes.com/cricket/virat-k...,Virat Kohli and AB De Villiers picked a combin...
6,WhatsApp increases group call limit from 4 peo...,https://www.hindustantimes.com/tech/whatsapp-i...,WhatsApp is now going to allow eight people to...
7,Himachal govt stressing on home-based teaching...,https://www.hindustantimes.com/education/himac...,"Amid lockdown in the wake of COVID-19, Himacha..."
8,"This Akshaya Tritiya, here’s how you can buy g...",https://www.hindustantimes.com/business-news/t...,If you are planning to buy gold on Akshaya Tri...
9,FIFA to give members $150m of virus relief funds,https://www.hindustantimes.com/football/fifa-t...,FIFA will release USD 150 million (139 million...


## Putting it all together in a function

In [3]:
def scrap_ht():
    t0 = time.time()
    df_features = pd.DataFrame(columns = ["title", "link", "Content"])
    
    #Coverpage operations
    url = "https://www.hindustantimes.com/top-news/"
    coverpage_request = requests.get(url)
    coverpage = coverpage_request.content
    coverpage_soup = BeautifulSoup(coverpage, "html5lib")

    #Extracting the big stories of the coverpage
    #Using custom fuction to avoid capturing <div class= "media-heading headingfour"
    bignews = coverpage_soup.find_all(lambda tag: tag.name == "div"
                                     and tag.get("class") == ["media-heading"])
    links = [item.find("a")["href"] for item in bignews]
    titles = [item.find("a").get_text() for item in bignews]
    #Scrapping individual articles starts here
    # The philosophy is we fetch links one by one and only the links that can scraped and the corresponding titls will be added
    #in the below lists
    final_articles = []
    final_links = []
    final_titles = []

    session = requests.session()

    for n in np.arange(len(links)):
        if "/liveupdates.hindustantimes.com/" in links[n]:
            continue
        time.sleep(0.5)
        try:
            article = session.get(links[n])
            article_content = article.content
            soup_article = BeautifulSoup(article_content, "html5lib")
            body = soup_article.find("div", class_ = "storyDetail")
            final_article = " ".join([p.get_text() for p in body.find_all("p")])
            final_articles.append(final_article)
            final_titles.append(titles[n])
            final_links.append(links[n])
            print(n, "done")
        except:
            print(n, "failed")
    
    #Adding df_bignews articles into the main dataframe df_features
    df_bignews = pd.DataFrame({"title" : final_titles, "link" : final_links, "Content" : final_articles})
    df_features = df_features.append(df_bignews, ignore_index = True)
    
    #Scraping rest of the articles on the coverpage
    othernews = coverpage_soup.find_all("a", class_ = "wclink2")
    
    links = [item["href"] for item in othernews]
    titles = [item.get_text() for item in othernews]    
    

    # The philosophy is we fetch links one by one and only the links that can scraped and the corresponding titls will be added
    #in the below lists
    final_articles = []
    final_links = []
    final_titles = []

    for n in np.arange(len(links)):
        #Accountinng for non-scrapable links
        if "/liveupdates.hindustantimes.com/" in links[n]:
            continue
        time.sleep(0.5)
        try:
            article = session.get(links[n])
            article_content = article.content
            soup_article = BeautifulSoup(article_content, "html5lib")
            body = soup_article.find("div", class_ = "storyDetail")
            final_article = " ".join([p.get_text() for p in body.find_all("p")])
            final_articles.append(final_article)
            final_titles.append(titles[n])
            final_links.append(links[n])
            print(n, "done")
        except:
            print(n, "failed")
    
    #Adding these articles to mail dataframe df_features
    df_othernews = pd.DataFrame({"title" : final_titles, "link" : final_links, "Content" : final_articles})
    df_features = df_features.append(df_othernews, ignore_index=True)
    print("time taken: {}".format(time.time()-t0))
    return df_features

In [4]:
df_features = scrap_ht()

0 done
1 done
2 done
3 done
4 done
5 done
6 done
7 done
8 done
9 done
0 done
1 done
3 done
4 done
5 done
6 done
7 done
8 done
9 done
10 done
11 done
12 done
13 done
14 done
15 done
16 done
17 done
18 done
19 done
20 done
21 done
22 done
23 done
24 done
25 done
26 done
27 done
28 done
29 done
30 done
31 done
32 done
33 failed
34 done
35 done
36 done
37 done
38 done
39 done
time taken: 34.090832233428955


In [9]:
df_features.to_csv("ht_scraped.csv", index=False)